# AI Agent Bootcamp for Beginners : LLM + API Call
# In this example we will create a 'Weather' agent that leverages LLM (OpenAI ) and API ( Weather API ) to fetch weather data real time

In [18]:
# ---------------------------------------------
# 🧊 STEP 1: Import necessary Python libraries
# ---------------------------------------------

import os                    # For interacting with environment variables (e.g., API keys)
import requests              # To make HTTP requests to weather APIs
from google.colab import userdata  # Colab-specific module to securely store user API keys

In [19]:
# ---------------------------------------------
# 🔐 STEP 2: Load API keys securely from Colab
# ---------------------------------------------
# These keys are used to access OpenAI and Weather API services.
# Get your OpenAI API from https://platform.openai.com/api-keys and set it as environment variable
# Get your Weather API from https://www.weatherapi.com/ and set it as environment variable

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["WEATHER_API_KEY"] = userdata.get("WEATHER_API_KEY")

In [20]:
# ---------------------------------------------
# 🌦️ STEP 3: Define a function to get weather info
# ---------------------------------------------

def get_weather(location):
    """
    Fetches current weather data for a given location using WeatherAPI.

    Args:
        location (str): The name of the city or place (e.g., "London", "New York")

    Returns:
        dict: Weather data in JSON format if successful.

    Raises:
        Exception: If the API key is missing or the HTTP request fails.
    """
    try:
        # Get the API key from environment variable
        api_key = os.environ.get("WEATHER_API_KEY")
        if not api_key:
            raise Exception("WEATHER_API_KEY is not set in environment variables.")

        # Set the WeatherAPI endpoint
        url = "http://api.weatherapi.com/v1/current.json"

        # Add parameters for the API request
        params = {
            "key": api_key,     # Your API key
            "q": location,      # The location to search weather for
            "aqi": "no"         # Disable Air Quality Index to keep response small
        }

        # Make a GET request to the Weather API
        response = requests.get(url, params=params)

        # Raise an error if the HTTP request was unsuccessful (e.g., 404 or 500)
        response.raise_for_status()

        # Return the data in JSON format (as a Python dictionary)
        return response.json()

    except requests.exceptions.RequestException as e:
        # Handle any errors that happened during the API call
        raise Exception(f"Weather API error: {str(e)}")


In [21]:
# ---------------------------------------------
# 🧪 STEP 4: Test the weather function
# ---------------------------------------------

# This block will only run if this script is the "main" file being executed
if __name__ == "__main__":
    try:
        # Call the function to get weather for London
        weather_data = get_weather("London") #provide city name here

        # Nicely print the results from the JSON response
        print(f"🌍 Weather in {weather_data['location']['name']}:")
        print(f"🌡️ Temperature: {weather_data['current']['temp_c']}°C")
        print(f"🌥️ Condition: {weather_data['current']['condition']['text']}")

    except Exception as e:
        # If anything goes wrong, print the error message
        print(f"❌ Error: {e}")


🌍 Weather in London:
🌡️ Temperature: 21.0°C
🌥️ Condition: Partly cloudy


In [22]:
# ---------------------------------------------
# 🧠 STEP 5: Import OpenAI and JSON modules
# ---------------------------------------------

import openai   # To access OpenAI's GPT models
import json     # To work with JSON (used for parsing arguments)


In [23]:
# ---------------------------------------------
# 🧾 STEP 6: Create agent weather_agent
# ---------------------------------------------
#
# The agent takes user query and then leverages LLM (openai) to decide if it need to invoke openai functions ( which is tagged to get_weather)
def weather_agent(user_query):
    """
    Conversational weather agent using OpenAI + WeatherAPI.

    Args:
        user_query (str): A question from the user (e.g., "What's the weather in Tokyo?")

    Returns:
        str: Formatted weather response or error message
    """
    try:
        # Step 1: Set up the OpenAI function tool list
        functions = [
            {
                "name": "get_weather",
                "description": "Get current weather for a location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "City name or location"
                        }
                    },
                    "required": ["location"]
                }
            }
        ]

        # Step 2: Create OpenAI client
        client = openai.OpenAI()

        # Step 3: Ask GPT the user question
        response = client.chat.completions.create(
            model="gpt-4-0613",
            messages=[{"role": "user", "content": user_query}],
            functions=functions,
            function_call="auto"
        )

        # Step 4: Check if GPT wants to call a function
        message = response.choices[0].message
        if message.function_call:
            func_name = message.function_call.name
            args = json.loads(message.function_call.arguments)

            if func_name == "get_weather":
                location = args.get("location")
                print(f"📍 GPT chose to get weather for: {location}")
                weather_data = get_weather(location)

                # Format and return a clean weather summary
                reply = (
                    f"🌤 Weather in {weather_data['location']['name']}:\n"
                    f"🌡 Temperature: {weather_data['current']['temp_c']}°C\n"
                    f"🌥 Condition: {weather_data['current']['condition']['text']}"
                )
                return reply
            else:
                return "⚠️ GPT wanted to call an unknown function."
        else:
            # If GPT didn't call a function, just return its answer
            return message.content

    except Exception as e:
        return f"❌ Error: {str(e)}"


In [24]:
# 🧪 Try asking a weather question
print(weather_agent("Is it hot in Mumbai today?")) #Enter sample question here with city name

# 🧪 Try another city
print(weather_agent("What's the weather like in Paris right now?")) #Enter sample question here with city name

# 🧪 Try something GPT might just answer directly (no function call)
print(weather_agent("How does weather forecasting work?"))


📍 GPT chose to get weather for: Mumbai
🌤 Weather in Mumbai:
🌡 Temperature: 27.3°C
🌥 Condition: Light rain
📍 GPT chose to get weather for: Paris
🌤 Weather in Paris:
🌡 Temperature: 22.2°C
🌥 Condition: Partly cloudy
Weather forecasting involves predicting the state of the atmosphere at a specific location over a certain period of time. This process is achieved through the collection of quantitative data about the current state of the atmosphere and understanding the laws of atmospheric behavior. Here are the key steps involved:

1. Data Collection: This involves gathering information about the current state of the atmosphere, including temperature, humidity, wind speed and direction, pressure, and other factors, from all over the world. These data are collected through weather stations, ships, buoys, aircraft, radars, satellites, and other observing systems.

2. Data Analysis: After the data has been collected, it is analyzed to identify atmospheric patterns and trends. This analysis incl